In [1]:
import os
import time
import pandas as pd
import numpy as np
import scipy as sc
import scipy.misc
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageDraw, ImageFont

In [2]:
# List of characters and classes
characterList = np.array('الف ب پ ت ث ج چ ح خ د ذ ر ز ژ س ش ص ض ط ظ ع غ ف ق ک گ ل م ن و ه ی'.split(' '))
numbersList = np.array('۰ ۱ ۲ ۳ ۴ ۵ ۶ ۷ ۸ ۹'.split(' '))
func = np.vectorize(str)
classesText = np.concatenate([np.array(['background']), func(numbersList), characterList])

In [3]:
# function for setup repetitive setup images
def setupPNGCharacter(coordinates, character, width, height, fontSize, font = "BNazanin.ttf",
    destination = 'E:/Hamed/Projects/Python/License Plate Detection/License-Plate-Detection/PersianCharactersList/{}.png'):
    if type(coordinates) != tuple:
        raise TypeError('Coordinates must be tuple')
    font = ImageFont.truetype(font, fontSize)
    img = Image.new('RGB', (width, height), color = (255, 255, 255))
    draw = ImageDraw.Draw(img)
    draw = draw.text(coordinates, character, fill = (0, 0, 0), font = font)
    img.save(destination.format(character))

In [4]:
# Create PNG files of all persian characters
def createAllPNG(listOfCharacter):
    if len(listOfCharacter) == 10:
        for number in range(len(listOfCharacter)):
            if number == 0:
                setupPNGCharacter((-2, -14), listOfCharacter[number], width = 54, height = 82,
                                  font = "BNazanin.ttf", fontSize = 130)
                setupPNGCharacter((-2, -11), listOfCharacter[number], width = 45, height = 70,
                                  font = "BNazanin.ttf", fontSize = 110,
                                  destination = 'E:/Hamed/Projects/Python/License Plate Detection/License-Plate-Detection/PersianCharactersList/smaller{}.png')
            else:
                setupPNGCharacter((-3, -10), listOfCharacter[number], width = 54, height = 82,
                                  font = "BNazanin.ttf", fontSize = 130)
                setupPNGCharacter((-3, -8), listOfCharacter[number], width = 45, height = 70,
                                  font = "BNazanin.ttf", fontSize = 110,
                                 destination = 'E:/Hamed/Projects/Python/License Plate Detection/License-Plate-Detection/PersianCharactersList/smaller{}.png')
    else:
        for characterIndex in range(1, len(listOfCharacter)):
            setupPNGCharacter((-3, -5), listOfCharacter[characterIndex], width = 90, height = 95,
                                  font = "BNazanin.ttf", fontSize = 100)

In [5]:
createAllPNG(numbersList)
createAllPNG(characterList)

In [6]:
# for prepare background and foreground image
def imageManager(character, width, height, nameOfSavedFile,
                  foregroundImage = 'PersianCharacterslist/{}.png',
                  backgroundImage = 'Background.png'):
    background = Image.open(backgroundImage)
    foreground = Image.open(foregroundImage.format(character))
    background.paste(foreground, (width, height))
    background.save(nameOfSavedFile)

In [7]:
# 1. Create images of all characters in different positions
# 2. Create summary of files
# 3. Create csv files in format image name, xmin, ymin, xmax, ymax, ClassID

In [8]:
# for finding character and it's coordinates
def searchAndAppendCharacter(character, name, xmin, xmax, ymin, ymax):
    classOfChracter = np.where(classesText == character)[0][0]
    xmin = xmin
    ymin = ymin
    xmax = xmax
    ymax = ymax
    return [name, xmin, ymin, xmax, ymax, classOfChracter]

In [9]:
def createSingelCharacter(widthPeriod, heightPeriod, imageNumber, kind, number, imageWidth, imageHeight,
                          listOfInformation, **kwargs):
    width = np.random.choice(np.arange(widthPeriod[0], widthPeriod[1]))
    height = np.random.choice(np.arange(heightPeriod[0], heightPeriod[1]))
    imageName = '{}.png'.format(imageNumber)
    imageManager (classesText[number], width, height, 'charOnPlates/{}/{}'.format(kind, imageName), **kwargs)
    listOfInformation = listOfInformation.append(searchAndAppendCharacter(classesText[number], imageName,
                                                width,width + imageWidth,height, height + imageHeight))

In [10]:
# for numbers
# width = 61 to 406
# heigth = 8 to 38
def createRandomCharactersInPlates(kind, numberOfNumbers, numberOfCharacters):
    if numberOfNumbers < 15:
        raise ValueError('numberOfNumbers must be greater than 15')
    listOfInformation = []
    imageNumber = 0
    for number in range(1, 11):
        for step in range(numberOfNumbers - 10):
            createSingelCharacter((61, 407), (8, 39), imageNumber, kind, number, 54, 82, listOfInformation)
            imageNumber += 1
# for smaller numbers
# width = 465 to 545
# heigth = 30 to 49
        for step in range(numberOfNumbers - 10, numberOfNumbers):
            createSingelCharacter((465, 546), (30, 50), imageNumber, kind, number, 45, 70, listOfInformation,
                                  foregroundImage = 'PersianCharacterslist/smaller{}.png')
            imageNumber += 1
# for characters
# width = 61 to 370
# heigth = 8 to 26
    imageNumber = numberOfNumbers * 10
    for number in range(12, 43):
        for step in range(numberOfCharacters):
            createSingelCharacter((61, 371), (8, 27), imageNumber, kind, number, 90, 95, listOfInformation)
            imageNumber += 1
    return listOfInformation

In [15]:
def createSamplesAndCsv(trainSize, testSize):
    trainInformation = createRandomCharactersInPlates('train', int(trainSize / 5 * 4), int(trainSize / 5))
    pd.DataFrame(trainInformation).to_csv('train.csv', header = None)
    testInformation = createRandomCharactersInPlates('test', int(testSize / 5 * 4), int(testSize / 5))
    pd.DataFrame(testInformation).to_csv('test.csv', header = None)